In [44]:
import frontmatter
import re
import urllib
import requests

DIRNAME = os.path.abspath('')
WWW_SHIPS_DATA_PATH = os.path.join(DIRNAME, "../../www/views/ships")
WWW_IMG_PATH = os.path.join(DIRNAME, "../../www/assets/img")

In [56]:
for ship_filename in os.listdir(WWW_SHIPS_DATA_PATH):
    md_match = re.match(r"(.*)\.md", ship_filename)
    if not md_match:
        continue
    ship_slug = md_match.groups()[0]
    ship_frontmatter = frontmatter.load(os.path.join(WWW_SHIPS_DATA_PATH, ship_filename))
    wikipedia_url = ship_frontmatter.metadata.get("wikipediaUrl")
    photo_path = ship_frontmatter.metadata.get("photo", "")
    if photo_path != "" or not wikipedia_url:
        continue
    parsed = urllib.parse.urlparse(wikipedia_url)
    country, title = re.match(r".*([a-z]{2,3})\.wikipedia\.org\/wiki\/(.*)", wikipedia_url).groups()
    json_url = f"http://{country}.wikipedia.org/w/api.php?action=query&titles={title}&prop=pageimages&format=json&pithumbsize=500"
    data = requests.get(json_url).json()
    keys = list(data["query"]["pages"])
    image_url = data["query"]["pages"][keys[0]].get("thumbnail", {}).get("source")
    if not image_url:
        print(f"could not find image url in {json_url}")
        continue
    img_ext = image_url[-10:].split(".")[-1]
    img_filename = f"{ship_slug}.{img_ext}"
    with open(os.path.join(f"{WWW_IMG_PATH}", img_filename), "wb") as f:
        f.write(requests.get(image_url).content)
    print(f"wrote image www/assets/js/{img_filename}")
    ship_frontmatter.metadata["photo"] = f"/img/{img_filename}"
    with open(os.path.join(WWW_SHIPS_DATA_PATH, ship_filename), "w") as f:
        f.write(frontmatter.dumps(ship_frontmatter))
    print(f"wrote back md to www/views/ships/{ship_filename}")
        

could not find image url in http://en.wikipedia.org/w/api.php?action=query&titles=Le_Bougainville&prop=pageimages&format=json&pithumbsize=500
wrote image www/assets/js/oosterdam-9221281.JPG
wrote back md to www/views/ships/oosterdam-9221281.md
wrote image www/assets/js/hamburg-9138329.JPG
wrote back md to www/views/ships/hamburg-9138329.md
wrote image www/assets/js/mazovia-9010814.jpg
wrote back md to www/views/ships/mazovia-9010814.md
wrote image www/assets/js/finlaggan-9482902.jpg
wrote back md to www/views/ships/finlaggan-9482902.md
wrote image www/assets/js/celestyal-crystal-7827213.jpg
wrote back md to www/views/ships/celestyal-crystal-7827213.md
wrote image www/assets/js/fast-ferries-andros-8907280.png
wrote back md to www/views/ships/fast-ferries-andros-8907280.md
wrote image www/assets/js/carnival-pride-9223954.jpg
wrote back md to www/views/ships/carnival-pride-9223954.md
wrote image www/assets/js/ocean-diamond-7325629.JPG
wrote back md to www/views/ships/ocean-diamond-7325629